# <center> <img src="../../labs/img/ITESOLogo.png" alt="ITESO" width="480" height="130"> </center>
# <center> **Departamento de Electrónica, Sistemas e Informática** </center>
---
### <center> **Procesamiento de Datos Masivos** </center>
---
### <center> **Primavera 2025** </center>
---
**Primer Examen**

**Fecha**: 14 de Marzo del 2025

**Nombre del estudiante**: Ana Cristina Luna Arellano

**Professor**: Pablo Camarillo Ramirez

In [6]:
import findspark
findspark.init()

In [7]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("SparkSQL-Exam-1-CristinaLuna  ") \
    .master("spark://d9c3cc2bade8:7077") \
    .config("spark.ui.port","4040") \
    .getOrCreate()
sc = spark.sparkContext

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/03/14 15:23:58 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
25/03/14 15:24:36 WARN TransportClientFactory: DNS resolution succeed for d9c3cc2bade8/172.21.0.2:7077 took 3009 ms


### Read the Data

In [16]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, from_json, when, avg, desc, year
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, DoubleType, DateType
from pyspark.sql.functions import current_date


df_employees = spark.read.option("header", "true").csv("/home/jovyan/notebooks/data/exam_P2025_ESI3914B/employees.csv")
df_departments = spark.read.option("header", "true").csv("/home/jovyan/notebooks/data/exam_P2025_ESI3914B/departments.csv")



### Extract the employee info from the JSON column (20 points):
    Extract the following the columns: name (string), department_id (integer), salary (double), and hire_date (date) from the employee_info column

In [9]:
schema = StructType([
    StructField("name", StringType(), True),
    StructField("department_id", IntegerType(), True),
    StructField("salary", DoubleType(), True),
    StructField("hire_date", DateType(), True)
])

df_employees = df_employees.withColumn("employee_info", from_json(col("employee_info"), schema))
df_employees = df_employees.select(
    col("employee_info.name"),
    col("employee_info.department_id"),
    col("employee_info.salary"),
    col("employee_info.hire_date")
)

df_employees.show(truncate=False)

+---------+-------------+---------+----------+
|name     |department_id|salary   |hire_date |
+---------+-------------+---------+----------+
|Caitlyn  |103          |115959.78|2002-06-10|
|Rachel   |104          |100820.16|2009-07-01|
|Carrie   |105          |114421.44|1998-12-10|
|Renee    |104          |54688.13 |1995-03-17|
|Gabriella|109          |106267.03|1995-02-09|
|Kristen  |101          |88237.54 |2010-11-15|
|Jonathan |102          |39323.42 |2012-06-30|
|Michelle |101          |64262.85 |2005-10-30|
|Michelle |105          |103521.88|1991-07-10|
|Lisa     |110          |55435.93 |2016-03-25|
|Cheryl   |103          |88073.75 |2020-08-21|
|Mikayla  |107          |95192.05 |2022-05-13|
|Lisa     |104          |36032.49 |2019-05-16|
|Sean     |108          |64904.69 |2021-06-06|
|Monica   |105          |92589.97 |2022-05-09|
|Katelyn  |104          |147225.58|2008-03-14|
|Linda    |108          |146632.64|2002-09-04|
|Tammy    |109          |128860.4 |2014-04-18|
|William  |10

### Join Operations (10 points):
    Join the employees DataFrame with the departments DataFrame on department_id to enrich the employee data with department details.

In [10]:
df_enriched = df_employees.join(df_departments, "department_id", "left")
df_enriched.show(truncate=False)


+-------------+---------+---------+----------+-------------------------------------------+-------------+
|department_id|name     |salary   |hire_date |department_name                            |location     |
+-------------+---------+---------+----------+-------------------------------------------+-------------+
|103          |Caitlyn  |115959.78|2002-06-10|Sales and Marketing                        |Chicago      |
|104          |Rachel   |100820.16|2009-07-01|Data Engineering                           |Zapopan      |
|105          |Carrie   |114421.44|1998-12-10|Data Science                               |Seattle      |
|104          |Renee    |54688.13 |1995-03-17|Data Engineering                           |Zapopan      |
|109          |Gabriella|106267.03|1995-02-09|Customer Service                           |San Francisco|
|101          |Kristen  |88237.54 |2010-11-15|Human Resources                            |San Diego    |
|102          |Jonathan |39323.42 |2012-06-30|Finance a

### Transformations (Using when()) (10 points):

In [11]:
df_enriched = df_enriched.withColumn("salary_category", when(col("salary") >= 55000, "High").otherwise("Low"))
df_enriched.select("name", "salary", "salary_category").show(truncate=False)


+---------+---------+---------------+
|name     |salary   |salary_category|
+---------+---------+---------------+
|Caitlyn  |115959.78|High           |
|Rachel   |100820.16|High           |
|Carrie   |114421.44|High           |
|Renee    |54688.13 |Low            |
|Gabriella|106267.03|High           |
|Kristen  |88237.54 |High           |
|Jonathan |39323.42 |Low            |
|Michelle |64262.85 |High           |
|Michelle |103521.88|High           |
|Lisa     |55435.93 |High           |
|Cheryl   |88073.75 |High           |
|Mikayla  |95192.05 |High           |
|Lisa     |36032.49 |Low            |
|Sean     |64904.69 |High           |
|Monica   |92589.97 |High           |
|Katelyn  |147225.58|High           |
|Linda    |146632.64|High           |
|Tammy    |128860.4 |High           |
|William  |142645.41|High           |
|Jorge    |87587.51 |High           |
+---------+---------+---------------+
only showing top 20 rows



### Filter and Group

In [12]:
# Filtrar empleados según la categoría de salario
df_high_salary = df_enriched.filter(col("salary_category") == "High")
df_low_salary = df_enriched.filter(col("salary_category") == "Low")

df_high_salary.show(truncate=False)
df_low_salary.show(truncate=False)

+-------------+---------+---------+----------+-------------------------------------------+-------------+---------------+
|department_id|name     |salary   |hire_date |department_name                            |location     |salary_category|
+-------------+---------+---------+----------+-------------------------------------------+-------------+---------------+
|103          |Caitlyn  |115959.78|2002-06-10|Sales and Marketing                        |Chicago      |High           |
|104          |Rachel   |100820.16|2009-07-01|Data Engineering                           |Zapopan      |High           |
|105          |Carrie   |114421.44|1998-12-10|Data Science                               |Seattle      |High           |
|109          |Gabriella|106267.03|1995-02-09|Customer Service                           |San Francisco|High           |
|101          |Kristen  |88237.54 |2010-11-15|Human Resources                            |San Diego    |High           |
|101          |Michelle |64262.8

In [13]:
# Calcular el salario promedio por departamento
df_high_avg_salary = df_high_salary.groupBy("department_name").agg(avg("salary").alias("avg_salary"))
df_low_avg_salary = df_low_salary.groupBy("department_name").agg(avg("salary").alias("avg_salary"))

df_high_avg_salary.show(truncate=False)
df_low_avg_salary.show(truncate=False)

+-------------------------------------------+------------------+
|department_name                            |avg_salary        |
+-------------------------------------------+------------------+
|Corporate Strategy and Business Development|102741.38324414717|
|Sales and Marketing                        |100839.65275449108|
|Data Engineering                           |101626.29492163012|
|Research and Development                   |98714.3003086419  |
|Finance and Accounting                     |100731.07877887784|
|Customer Service                           |101585.01600000002|
|Legal                                      |99366.3129102167  |
|Data Science                               |101903.63710344829|
|Operations                                 |100169.65621722837|
|Human Resources                            |104999.43191489363|
+-------------------------------------------+------------------+



+-------------------------------------------+------------------+
|department_name                            |avg_salary        |
+-------------------------------------------+------------------+
|Corporate Strategy and Business Development|41590.741833333326|
|Sales and Marketing                        |41150.40277777778 |
|Data Engineering                           |41358.50794117647 |
|Finance and Accounting                     |42740.952888888874|
|Research and Development                   |41426.43521126761 |
|Customer Service                           |42644.472021276604|
|Legal                                      |41160.26616438357 |
|Data Science                               |41974.18958333334 |
|Operations                                 |40646.100705882345|
|Human Resources                            |41751.64784810126 |
+-------------------------------------------+------------------+



### Sort 

In [14]:
# Obtener los Top 5 empleados con salarios más altos
df_top5_high = df_high_salary.orderBy(desc("salary")).limit(5)
df_top5_low = df_low_salary.orderBy(desc("salary")).limit(5)

df_top5_high.show(truncate=False)
df_top5_low.show(truncate=False)

+-------------+---------+---------+----------+-------------------------------------------+-----------+---------------+
|department_id|name     |salary   |hire_date |department_name                            |location   |salary_category|
+-------------+---------+---------+----------+-------------------------------------------+-----------+---------------+
|101          |Gabriella|149989.73|2018-09-14|Human Resources                            |San Diego  |High           |
|101          |Katherine|149979.3 |2017-07-26|Human Resources                            |San Diego  |High           |
|110          |Ryan     |149963.1 |1990-07-03|Corporate Strategy and Business Development|Los Angeles|High           |
|107          |Caitlyn  |149956.54|2000-07-27|Legal                                      |Chicago    |High           |
|107          |Mark     |149915.56|2007-11-06|Legal                                      |Chicago    |High           |
+-------------+---------+---------+----------+--

+-------------+-----+--------+----------+-------------------------------------------+-------------+---------------+
|department_id|name |salary  |hire_date |department_name                            |location     |salary_category|
+-------------+-----+--------+----------+-------------------------------------------+-------------+---------------+
|110          |Linda|54993.53|2017-01-15|Corporate Strategy and Business Development|Los Angeles  |Low            |
|104          |Tammy|54991.71|2004-12-07|Data Engineering                           |Zapopan      |Low            |
|102          |Aaron|54989.45|2011-03-20|Finance and Accounting                     |New York     |Low            |
|101          |Craig|54945.2 |2016-04-24|Human Resources                            |San Diego    |Low            |
|109          |Aaron|54937.3 |1994-06-25|Customer Service                           |San Francisco|Low            |
+-------------+-----+--------+----------+-------------------------------

### Find the number of employees whit more years in the company

In [17]:
df_enriched = df_enriched.withColumn("years_in_company", year(current_date()) - year(col("hire_date")))

# empleados con más años en la empresa
df_most_years = df_enriched.orderBy(desc("years_in_company"))
df_count_most_years = df_most_years.count()

df_most_years.show(truncate=False)
print(f"Total empleados con más años en la empresa: {df_count_most_years}")

+-------------+--------+---------+----------+------------------------+-------------+---------------+----------------+
|department_id|name    |salary   |hire_date |department_name         |location     |salary_category|years_in_company|
+-------------+--------+---------+----------+------------------------+-------------+---------------+----------------+
|108          |Jonathan|137038.82|1988-07-26|Research and Development|Philadelphia |High           |37              |
|102          |Aaron   |107848.52|1988-12-02|Finance and Accounting  |New York     |High           |37              |
|109          |Jorge   |93917.58 |1988-06-19|Customer Service        |San Francisco|High           |37              |
|103          |Mark    |122033.27|1988-11-20|Sales and Marketing     |Chicago      |High           |37              |
|107          |William |113102.8 |1988-12-16|Legal                   |Chicago      |High           |37              |
|107          |Megan   |56472.42 |1988-04-03|Legal      

### Display the  Spark Plan

In [18]:
df_high_salary.explain()

== Physical Plan ==
AdaptiveSparkPlan isFinalPlan=false
+- Project [department_id#49, name#48, salary#50, hire_date#51, department_name#39, location#40, CASE WHEN (salary#50 >= 55000.0) THEN High ELSE Low END AS salary_category#110]
   +- BroadcastHashJoin [department_id#49], [cast(department_id#38 as int)], LeftOuter, BuildRight, false
      :- Project [employee_info#44.name AS name#48, employee_info#44.department_id AS department_id#49, employee_info#44.salary AS salary#50, employee_info#44.hire_date AS hire_date#51]
      :  +- Project [from_json(StructField(name,StringType,true), StructField(department_id,IntegerType,true), StructField(salary,DoubleType,true), StructField(hire_date,DateType,true), employee_info#18, Some(GMT)) AS employee_info#44]
      :     +- Filter ((from_json(StructField(salary,DoubleType,true), employee_info#18, Some(GMT)).salary >= 55000.0) <=> true)
      :        +- FileScan csv [employee_info#18] Batched: false, DataFilters: [((from_json(StructField(salary

In [19]:
spark.stop()